In [116]:
import pandas as pd
import numpy as np

breast_data = pd.read_csv('datasets/breast-w.csv')

cardio_data = pd.read_csv('datasets/Diabetes.csv')

In [117]:
breast_data.head(5)
breast_data.drop(columns=['subspaces'],inplace=True)
breast_data

,d1,d2,d3,d4,d5,d6,d7,d8,d9,is_anomaly
0,1,1,1,3,2,1,1,1,1,0
1,3,1,1,1,2,1,2,1,1,0
2,2,1,1,1,2,1,2,1,1,0
3,2,1,1,1,2,1,2,1,1,0
4,7,6,10,5,3,10,9,10,2,0
...,...,...,...,...,...,...,...,...,...,...
694,3,1,1,1,2,1,2,1,1,0
695,4,2,1,1,2,1,2,1,1,0
696,3,1,1,1,2,1,3,1,1,0
697,1,1,1,1,2,1,1,1,1,0


In [118]:
cardio_data.head(5)
cardio_data.drop(columns=['subspaces'],inplace=True)
cardio_data

,d1,d2,d3,d4,d5,d6,d7,d8,is_anomaly
0,2,75,64,24,55,29.7,0.370,33,0
1,2,120,76,37,105,39.7,0.215,29,0
2,13,129,0,30,0,39.9,0.569,44,0
3,9,72,78,25,0,31.6,0.280,38,0
4,11,155,76,28,150,33.3,1.350,51,0
...,...,...,...,...,...,...,...,...,...
763,6,92,92,0,0,19.9,0.188,28,0
764,2,112,78,50,140,39.4,0.175,24,0
765,8,126,74,38,75,25.9,0.162,39,0
766,3,125,58,0,0,31.6,0.151,24,0


Insert missing values to both datasets!

In [119]:
from generate_missing_values import generate_mcar_missing_values

# Find the feature's names.
cardio_cols = [i for i in list(cardio_data.columns) if i != 'is_anomaly']
breast_cols = [i for i in list(breast_data.columns) if i != 'is_anomaly']

#Only inser missing data at features!
#cardio_data.loc[:,cardio_cols] = generate_mcar_missing_values(cardio_data.loc[:,cardio_cols], 0.25).copy()

#breast_data.loc[:,breast_cols] = generate_mcar_missing_values(breast_data.loc[:,breast_cols], 0.25).copy()


In [133]:
import autosklearn 
from autosklearn.classification import AutoSklearnClassifier

from imputation_code.mm_preprocessor import MeanModePreprocessor
from imputation_code.scaler import ScalingPreprocessing
from imputation_code.no_preprocessor import NoPreprocessing

from sklearn.metrics import accuracy_score

autosklearn.pipeline.components.data_preprocessing.add_preprocessor(NoPreprocessing)
# Add NoPreprocessing component to auto-sklearn.
#autosklearn.pipeline.components.feature_preprocessing.add_preprocessor(MeanModePreprocessor)
autosklearn.pipeline.components.data_preprocessing.add_preprocessor(ScalingPreprocessing)

In [134]:
accuracy_scorer = autosklearn.metrics.make_scorer(
    name="accu",
    score_func=accuracy_score,
    optimum=1,
    greater_is_better=True,
    needs_proba=False,
    needs_threshold=False,
)

import autosklearn.classification
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train)

In [135]:
#"data_preprocessor": ['ScalingPreprocessing'],\
clf = AutoSklearnClassifier(
    time_left_for_this_task=300,
    # Bellow two flags are provided to speed up calculations
    # Not recommended for a real implementation
    initial_configurations_via_metalearning=0,
    smac_scenario_args={"runcount_limit": 10},
    ensemble_kwargs = {'ensemble_size': 1},
    resampling_strategy = 'cv',
    resampling_strategy_arguments ={"folds" :5},
)    #metric = accuracy_scorer,
clf.fit(cardio_data.loc[:,cardio_cols], cardio_data.loc[:,'is_anomaly'])


2023-11-09 13:48:31,109 - Client-EnsembleBuilder - WARNING - No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2023-11-09 13:48:31,109:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
2023-11-09 13:48:31,132 - Client-EnsembleBuilder - INFO - Deleting runs {Run(id=(1, 2, 0.0), losses={'accuracy': 0.05078125})} due to nbest=50 or worse than dummies
2023-11-09 13:48:31,152 - Client-EnsembleBuilder - INFO - Ensemble Selection:
	Trajectory: 0: 0.05078
	Members: [0]
	Weights: [1.]
	Identifiers: (1, 1, 0.0)

2023-11-09 13:48:31,181 - Client-EnsembleBuilder - INFO - Deleted files for Run(id=(1, 2, 0.0), losses={'accuracy': 0.05078125})
2023-11-09 13:48:31,243 - Client-EnsembleBuilder - INFO - DummyFuture: ([{'Timestamp': Timestamp('2023-11-09 13:48:31.172628'), 'ensemble_optimization_score': 0.94921875}], 50)/SingleThreadedClient() Started Ensemb

AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      ensemble_kwargs={'ensemble_size': 1},
                      initial_configurations_via_metalearning=0,
                      per_run_time_limit=30, resampling_strategy='cv',
                      resampling_strategy_arguments={'folds': 5},
                      smac_scenario_args={'runcount_limit': 10},
                      time_left_for_this_task=300)

In [136]:
print(clf.sprint_statistics())

auto-sklearn results:
  Dataset name: deadccab-7ef5-11ee-8877-dd78a1d2e07b
  Metric: accuracy
  Best validation score: 0.949219
  Number of target algorithm runs: 10
  Number of successful target algorithm runs: 5
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 0
  Number of target algorithms that exceeded the memory limit: 0



In [132]:
losses_and_configurations = [
    (run_value.cost, run_key.config_id)
    for run_key, run_value in clf.automl_.runhistory_.data.items()
]
losses_and_configurations.sort()
print("Lowest loss:", losses_and_configurations[0][0])
print(
    "Best configuration:",
    clf.automl_.runhistory_.ids_config[losses_and_configurations[0][1]],
)

Lowest loss: 0.050781250000000014
Best configuration: Configuration(values={
  'balancing:strategy': 'none',
  'classifier:__choice__': 'random_forest',
  'classifier:random_forest:bootstrap': 'True',
  'classifier:random_forest:criterion': 'gini',
  'classifier:random_forest:max_depth': 'None',
  'classifier:random_forest:max_features': 0.5,
  'classifier:random_forest:max_leaf_nodes': 'None',
  'classifier:random_forest:min_impurity_decrease': 0.0,
  'classifier:random_forest:min_samples_leaf': 1,
  'classifier:random_forest:min_samples_split': 2,
  'classifier:random_forest:min_weight_fraction_leaf': 0.0,
  'data_preprocessor:__choice__': 'feature_type',
  'data_preprocessor:feature_type:numerical_transformer:imputation:strategy': 'mean',
  'data_preprocessor:feature_type:numerical_transformer:rescaling:__choice__': 'standardize',
  'feature_preprocessor:__choice__': 'no_preprocessing',
})



In [ ]:
for run_key in clf.automl_.runhistory_.data:
    print("#########")
    print(run_key)
    print(clf.automl_.runhistory_.data[run_key])